In [46]:
import re
import os
import csv

In [47]:
csv_filename = 'tek.csv'
tek_directory = 'podatki'

In [48]:
def read_file_to_string(directory, filename):
    path = os.path.join(directory, filename)
    print(path)
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

In [49]:
def page_to_ads(page_content):
    vzorec = r'{"__typename":"ParticipantResult".+?"triLetterCode":"..."}}},'
    return re.findall(vzorec, page_content, flags=re.DOTALL)

In [50]:
def get_dict_from_ad_block(block):
    vzorec_mesto = r'"order":(.+?),"position":'
    vzorec_ime = r'{"__typename":"Athlete","name":"([\w -\.]+?)","surname"'
    vzorec_priimek = r',"surname":"([\w -\.]+?)","thumbnail'
    vzorec_cas = r'"type":"TIME","unit":"([\d.:]+?)"},"'
    vzorec_drzava = r'{"__typename":"Country","name":"([\w -\.]+?)","code"'
    try:
        mesto = re.search(vzorec_mesto, block).group(1)
        ime = re.search(vzorec_ime, block).group(1)
        priimek = re.search(vzorec_priimek, block).group(1)
        drzava = re.search(vzorec_drzava, block).group(1)
    except AttributeError:
        print(f"Nepopolni vzorci pri (čudnem?) oglasu\n{block}")
        raise
    try:
        cas = re.search(vzorec_cas, block).group(1)
        cas = cas[1:8]
    except AttributeError:
        cas = None
    def pretvorba(cas):
        ure = int(cas[0])
        minute = int(cas[2:4])
        sekunde = int(cas[5:7])
        return 3600 * ure + 60 * minute + sekunde
    sekunde = pretvorba(cas)
    return {"Mesto": mesto, "Ime": ime, "Priimek": priimek, "Čas": cas, "Sekunde": sekunde, 
    "Država": drzava}

In [51]:
def ads_from_file(filename, directory):
    vsebina = read_file_to_string(directory, filename)
    rezultati = page_to_ads(vsebina)
    rezultati_prvih_50 = rezultati[:50]
    slovarji = []
    for rezultat in rezultati_prvih_50:
        slovarji.append(get_dict_from_ad_block(rezultat))
    return slovarji

In [52]:
def write_csv(fieldnames, rows, directory, filename):
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)
    return

In [53]:
def write_results_to_csv(ads, directory, filename):
    assert ads and (all(slovar.keys() == ads[0].keys() for slovar in ads))
    imena_stolpcev = (ads[0])
    write_csv(imena_stolpcev, ads, directory, filename)

In [54]:
for i in range(5):
    if i == 0:
        vsi_slovarji = []
        seznam = ads_from_file('tek0', tek_directory)
        for j in range(50):
            seznam[j]['Leto'] = 2016
        vsi_slovarji += seznam
    if i == 1:
        seznam = ads_from_file('tek1', tek_directory)
        for j in range(50):
            seznam[j]['Leto'] = 2012
        vsi_slovarji += seznam
    if i == 2:
        seznam = ads_from_file('tek2', tek_directory)
        for j in range(50):
            seznam[j]['Leto'] = 2008
        vsi_slovarji += seznam
    if i == 3:
        seznam = ads_from_file('tek3', tek_directory)
        for j in range(50):
            seznam[j]['Leto'] = 2004
        vsi_slovarji += seznam
    if i == 4:
        seznam = ads_from_file('tek4', tek_directory)
        for j in range(50):
            seznam[j]['Leto'] = 2000
        vsi_slovarji += seznam

podatki\tek0
podatki\tek1
podatki\tek2
podatki\tek3
podatki\tek4


In [55]:
write_results_to_csv(vsi_slovarji, "obdelani_podatki", csv_filename)